# StackAI Vector Database Exploration

Interactive notebook for testing the StackAI vector database API.

**Setup:**
1. Start the server: `make start`
2. Seed test data: `python scripts/seed_data.py --library all`
3. Run the cells below

In [ ]:
import httpx

BASE_URL = "http://localhost:8000/api/v1"
client = httpx.Client(base_url=BASE_URL, timeout=30.0)

# Helper functions

def search(query: str, library: str = "recipes_lib", k: int = 3):
    """Search a library and display formatted results."""
    response = client.post(f"/libraries/{library}/search", json={"query": query, "k": k})
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return
    
    data = response.json()
    print(f'Query: "{data["query"]}"')
    print(f"Library: {library}")
    print(f"Results: {data['result_count']}")
    print("-" * 60)
    
    for i, result in enumerate(data["results"], 1):
        chunk = result["chunk"]
        print(f"\n[{i}] Score: {result['score']:.4f}")
        print(f"    Doc: {chunk['document_id']}")
        print(f"    Text: {chunk['text']}")


def list_libraries():
    """List all libraries."""
    response = client.get("/libraries")
    libraries = response.json()
    print(f"Libraries ({len(libraries)}):")
    for lib in libraries:
        print(f"  - {lib['id']}: {lib['name']}")


def list_documents(library: str):
    """List documents in a library."""
    response = client.get(f"/libraries/{library}/documents")
    if response.status_code != 200:
        print(f"Error: {response.text}")
        return
    docs = response.json()
    print(f"Documents in {library} ({len(docs)}):")
    for doc in docs:
        print(f"  - {doc['id']}: {doc['name']}")


def list_chunks(document: str, show_text: bool = True):
    """List chunks in a document."""
    response = client.get(f"/documents/{document}/chunks")
    if response.status_code != 200:
        print(f"Error: {response.text}")
        return
    chunks = response.json()
    print(f"Chunks in {document} ({len(chunks)}):")
    for chunk in chunks:
        if show_text:
            print(f"  [{chunk['id']}] {chunk['text'][:80]}..." if len(chunk['text']) > 80 else f"  [{chunk['id']}] {chunk['text']}")
        else:
            print(f"  - {chunk['id']}")


def health_check():
    """Check if the server is running."""
    try:
        response = httpx.get("http://localhost:8000/health")
        if response.status_code == 200:
            print("Server is running")
        else:
            print(f"Server returned: {response.status_code}")
    except httpx.ConnectError:
        print("Cannot connect to server. Start it with: make start")


print("Helper functions loaded: search(), list_libraries(), list_documents(), list_chunks(), health_check()")

## Quick Start

In [ ]:
health_check()

In [ ]:
list_libraries()

## Search

Available libraries (after seeding):
- `recipes_lib` - Cooking recipes
- `support_lib` - Support knowledge base
- `products_lib` - Product manuals

In [ ]:
search("How do I make a creamy pasta sauce?", library="recipes_lib")

In [ ]:
search("How do I reset my password?", library="support_lib")

In [ ]:
search("bluetooth pairing", library="products_lib", k=5)

In [ ]:
# Try your own query
search("chicken curry recipe", library="recipes_lib", k=5)

## Browse Data

In [ ]:
list_documents("recipes_lib")

In [ ]:
list_chunks("recipes_spaghetti_carbonara")

## Manual API Calls

For more control, use the client directly:

In [ ]:
# Raw API call example
response = client.post("/libraries/recipes_lib/search", json={
    "query": "baking cookies",
    "k": 2
})
response.json()